In [1]:
print("awan")

awan


In [2]:
pwd

'e:\\chatbot\\Chatbot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
pwd

'e:\\chatbot\\Chatbot'

In [5]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

e:\chatbot\Chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def load_pdf(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls= PyPDFLoader
    )

    document = loader.load()

    return document

In [7]:
pwd

'e:\\chatbot\\Chatbot'

In [8]:
extracted_doc = load_pdf("dataset")

In [9]:
len(extracted_doc)

93

In [10]:
from typing import List
from langchain_core.documents import Document

In [11]:
#extracted_doc

In [12]:
def extract_meta_data(docs: List[Document]) -> List[Document]:

    min_docs: List[Document] = []

    for doc in docs:
        src = doc.metadata.get("source")
        page = doc.metadata.get("page") 
        min_docs.append(
            Document(
                page_content=doc.page_content,
                metadata = {"source" : src, 
                            "page": page}

            )
        )
    return min_docs

In [13]:
minimal_doc = extract_meta_data(extracted_doc)

In [14]:
minimal_doc[:5]

[Document(metadata={'source': 'dataset\\agent_ai.pdf', 'page': 0}, page_content=''),
 Document(metadata={'source': 'dataset\\agent_ai.pdf', 'page': 1}, page_content='2\nMastering AI Agents\nPreface\nIn our previous e-book, “Mastering RAG,” our goal was clear: building enterprise-grade \nRAG systems, productionizing them, monitoring their performance, and improving them. \nAt the core of it, we understood how RAG systems enhance an LLM’s ability to work with \nspecific knowledge by providing relevant context. \nIn this e-book, we’re taking a step further and asking, “How do we use LLMs to \naccomplish end-to-end tasks?” This singular question opens up a door: AI agents. A RAG \nsystem helps an LLM provide accurate answers based on given context. An AI agent \ntakes that answer and actually does something with it — makes decisions, executes \ntasks, or coordinates multiple steps to achieve a goal.\nA RAG-enhanced LLM could help answer questions about policy details by pulling relevant \n

In [15]:
#Lets do a chunking


In [16]:
def chunking(minimal_doc):
    text_chunk = RecursiveCharacterTextSplitter(
        chunk_size=100,
        chunk_overlap=20
    )

    split_chunks = text_chunk.split_documents(minimal_doc)

    return split_chunks



In [17]:
chunked_doc = chunking(minimal_doc)

In [18]:
len(chunked_doc)

1507

In [19]:
from langchain_huggingface import HuggingFaceEmbeddings

In [20]:
def download_model():
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name = model_name
        #kwargs={"device" : "cpu"}
    )

    return embeddings

In [21]:
embedded_model = download_model()

In [22]:
print(embedded_model)

model_name='sentence-transformers/all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} query_encode_kwargs={} multi_process=False show_progress=False


In [23]:
x = embedded_model.embed_query("I am Haris!")
len(x)

384

In [24]:
from dotenv import load_dotenv
load_dotenv()

True

In [62]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")


In [63]:
print("Pinecone:", PINECONE_API_KEY is not None)
print("OpenRouter:", OPENROUTER_API_KEY is not None)

Pinecone: True
OpenRouter: True


In [ ]:
from pinecone import Pinecone

In [65]:
pine_api = PINECONE_API_KEY
openrouter_api_key = OPENROUTER_API_KEY

In [28]:
pine_api

'pcsk_6Zqrci_TTmjJX51jtHUoNxHHhH4rwG94k96AKsmd4cLHvwgv7HP6K38YTVN1Luehnb9W6g'

In [29]:
#Lets verify the api key 

In [30]:
pc = Pinecone(api_key=pine_api)

In [31]:
print(pc)

In [37]:
from pinecone import ServerlessSpec
index_name = "chatbot-medical"

In [38]:
if not pc.has_index(index_name):
    pc.create_index(
        name= index_name,
        dimension=384,
        metric="cosine",
        spec = ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )


index = pc.Index(index_name)

In [40]:
#Storing into the Vector Database

In [41]:
from langchain_pinecone import PineconeVectorStore

In [43]:
storing = PineconeVectorStore.from_documents(
    documents= chunked_doc,
    embedding=embedded_model,
    index_name=index_name
)

In [44]:
#Retriver Pipeline

In [46]:
retriveing = storing.as_retriever(search_type = "similarity",
                     search_kwrags = {"k" : 3})

In [47]:
retriveing.invoke("What is AI")

[Document(id='3c235a2f-369d-411c-8cdb-48727104c41a', metadata={'page': 7.0, 'source': 'dataset\\agent_ai.pdf'}, page_content='once it’s on its way.”\nWhat are AI agents?'),
 Document(id='5c95cc6b-e7a0-4e64-9433-c0fa2210ccac', metadata={'page': 91.0, 'source': 'dataset\\agent_ai.pdf'}, page_content='an AI agent›s request to'),
 Document(id='5a707bf3-17a0-4842-ba2d-91c0f1efe59f', metadata={'page': 9.0, 'source': 'dataset\\agent_ai.pdf'}, page_content='Now that we’re familiar with what AI agents are, let’s look at different types of AI'),
 Document(id='c0285627-7a5d-4018-a20d-58a4f8d692d0', metadata={'page': 6.0, 'source': 'dataset\\agent_ai.pdf'}, page_content='WHAT ARE AI \nAGENTS?\n01 \nCHAPTER')]

In [76]:
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

In [78]:
llm = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
    model="alibaba/tongyi-deepresearch-30b-a3b:free",
)

In [68]:
system_prompt = (
    "You are an expert AI Agent"
    "You need to answer my question and if you dont know, just say I dont know"
    "{context}"
)

In [72]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system" , system_prompt),
        ("human" , "{input}")
    ]
)

#Just to make sure that chat model takes two prompts 

In [80]:
qa_chain = create_stuff_documents_chain(llm , prompt)
rag_chain = create_retrieval_chain(retriveing , qa_chain)

In [82]:
response = rag_chain.invoke({"input": "What is AI"})

In [83]:
print(response["answer"])



Artificial Intelligence (AI) refers to the simulation of human intelligence in machines. These systems are designed to learn from experience, adapt to new inputs, and perform tasks typically requiring human intelligence.

### Key Characteristics of AI:
1. **Learning**: AI systems can learn from data, experiences, and interactions with their environment.

2. **Adaptation**: AI systems can adapt to new inputs and environments, allowing them to perform tasks in a wide range of settings.

3. **Reasoning**: AI systems can reason about tasks and decisions, allowing them to make decisions and perform tasks in a more intelligent and efficient manner.

4. **Perception**: AI systems can perceive their environment through sensors or other means, allowing them to interact with and respond to their environment in a more intelligent and efficient manner.

### Types of AI:

1. **Reactive AI**: This type of AI is designed to perform specific tasks based on predefined rules and data. Reactive AI syst